In [4]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

from langchain_core.output_parsers import StrOutputParser
import langchain
# load_dotenv(dotenv_path='.env')
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")



In [2]:
print(f"lang chain.version : {langchain.__version__}")
print(f"Open API Key : {OPENAI_API_KEY[:10]}")

lang chain.version : 0.3.27
Open API Key : gsk_yRNqBi


In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# LLM 설정
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="openai/gpt-oss-120b",
    temperature=0.7
)

def create_attraction_recommendation_chain():
    """여행지에서 대표 명소를 추천하는 체인"""
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 세계 각국의 관광 명소에 대해 잘 알고 있는 여행 전문가입니다."),
        ("system", "사용자가 입력한 도시나 국가에서 가장 대표적이고 유명한 관광 명소 1가지만 추천해주세요."),
        ("system", "명소의 이름만 간단하게 답변하세요. 설명은 하지 마세요."),
        ("user", "{destination}")
    ])
    
    return prompt | llm | StrOutputParser()

def create_attraction_detail_chain():
    """명소에 대한 상세 정보를 제공하는 체인"""
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 관광 명소에 대한 상세한 정보를 제공하는 전문 가이드입니다."),
        ("system", "다음 형식으로 명소에 대한 정보를 제공해주세요:"),
        ("system", "명소: [명소 이름]"),
        ("system", "역사: [역사적 배경과 건설/형성 과정]"),
        ("system", "특징: [주요 특징과 볼거리]"),
        ("system", "방문 팁: [실용적인 방문 정보와 팁]"),
        ("user", "{attraction}")
    ])
    
    return prompt | llm | StrOutputParser()


attraction_recommendation_chain = create_attraction_recommendation_chain()
attraction_detail_chain = create_attraction_detail_chain()

print("관광 명소 추천 2단계 체인이 생성되었습니다!")


관광 명소 추천 2단계 체인이 생성되었습니다!


In [13]:
def travel_recommendation_system(destination: str):
    # 1단계: 명소 추천
    attraction_name = attraction_recommendation_chain.invoke({"destination": destination})
    
    # 2단계: 상세 정보
    attraction_detail = attraction_detail_chain.invoke({"attraction": attraction_name})
    
    return {
        "추천 명소": attraction_name,
        "상세 정보": attraction_detail
    }

# 실행 예시
result1 = travel_recommendation_system("블라디보스톡")
print(result1)


{'추천 명소': '골든 브리지', '상세 정보': '**명소:** 골든 브리지 (Cầu Vàng) – 베트남 다낭 호이안 인근 바나 힐스 (Bà Nà Hills)  \n\n---\n\n### 역사\n- **구상·계획:** 2017년 베트남의 유명 리조트·관광 개발 기업인 **Sun Group**이 ‘바나 힐스’ 리조트 단지 내 새로운 랜드마크를 만들기 위해 구상. ‘천국에 닿은 길’이라는 컨셉으로, 방문객이 마치 구름 위를 걷는 듯한 경험을 제공하고자 함.  \n- **설계·시공:** 프랑스 건축가 **마르크 푸르노(Marc Forné)**와 베트남 현지 건축·엔지니어 팀이 협업. 2018년 3월 착공, 2018년 12월에 완공돼 2019년 3월 공식 개장.  \n- **구조적 특징:** 전체 길이 150\u202fm, 높이 140\u202fm(해발 고도)이며, 두 개의 거대한 손 모양 구조물이 다리를 받치고 있다. 손은 스틸 프레임에 석고와 석회석을 입혀 마감했으며, 2\u202f톤(≈2,000\u202fkg) 무게.  \n- **문화적 의미:** ‘두 손이 세상을 잡고 있다’는 의미를 담아, 인류가 자연과 조화롭게 살아가길 바라는 메시지를 전달한다. 개장 이후 SNS와 언론에 크게 부각돼 베트남을 대표하는 현대 건축물·관광 명소로 자리 잡았다.\n\n---\n\n### 특징\n| 구분 | 내용 |\n|------|------|\n| **전경** | 구름 위를 걷는 듯한 파노라마 뷰. 다리 위에서 다낭 시내, 바다, 그리고 주변 산맥을 360°로 조망 가능. |\n| **손 조형물** | 30\u202fm 높이·30\u202fm 길이의 거대한 손 두 개가 다리를 받치고 있음. 석회석 마감으로 햇빛에 은은히 빛나며, 밤에는 LED 조명으로 색다른 분위기 연출. |\n| **재질·구조** | 고강도 스틸 프레임 + 석고·석회석 외피. 내진 설계 적용(지진 등 자연재해 대비). |\n| **주변 시설** | 바나 힐스 케이블카(세계 최장·최고 고도), 프랑스 마

In [16]:


result1 = travel_recommendation_system("블라디보스톡")


In [17]:
print(result1)

{'추천 명소': '러시키 다리', '상세 정보': '**명소:** 러시키 다리(※ 정확한 명칭·위치가 확인되지 않아, 추가 정보를 부탁드립니다.)\n\n**역사:** 현재 알려진 자료에서는 “러시키 다리”라는 이름의 다리에 대한 구체적인 역사적 배경이 확인되지 않고 있습니다. 다리 이름이 현지어(예: 러시아어, 우즈베크어 등) 혹은 로마자 표기에서 변형된 경우가 많아, 정확한 명칭이나 위치를 알려주시면 보다 상세한 건설·형성 과정을 찾아드릴 수 있습니다.\n\n**특징:** 이름만으로는 어떤 종류의 다리인지(교량·보행자 전용·관광용 등)와 주변 풍경(강, 호수, 산맥 등)을 파악하기 어렵습니다. 일반적으로 관광 명소로 알려진 다리들은 다음과 같은 특징을 가집니다.  \n\n- **구조적 아름다움**: 현대식 케이블 스테이 또는 전통적인 아치형 구조 등  \n- **전망 포인트**: 다리 위에서 주변 경관을 한눈에 조망할 수 있음  \n- **문화·역사적 의미**: 지역의 교통·경제·역사와 깊은 연관이 있음  \n\n**방문 팁:**  \n1. **위치 확인**: 정확한 주소나 근처 주요 도시·관광지를 파악한 뒤 네비게이션에 입력하세요.  \n2. **교통수단**: 대중교통(버스·지하철·택시) 이용이 가능한 경우가 많으며, 자가용을 이용한다면 주차 공간을 미리 확인하세요.  \n3. **베스트 타임**: 일출·일몰 시간대는 조명이 아름다워 사진 촬영에 최적입니다. 날씨가 맑은 날을 선택하면 더욱 좋은 경관을 감상할 수 있습니다.  \n4. **준비물**: 편한 보행화·선크림·물병·카메라(또는 스마트폰) 등을 챙기세요.  \n5. **주변 관광**: 다리 주변에 전통 시장·박물관·자연공원 등이 있다면 함께 방문하면 여행 효율이 높아집니다.  \n\n---\n\n**※ 추가 정보 요청**  \n러시키 다리의 정확한 위치(예: 국가·도시·지역)나 다른 알려진 명칭이 있다면 알려 주세요. 그러면 해당 다리의 역사·특징·방문 팁을 보다 구체적으로 제공해 드릴 수 있습니